# TP4
## Grupo 15

Carlos Eduardo Da Silva Machado A96936

Gonçalo Manuel Maia de Sousa A97485

## Exercício 4.2

### Descrição do Problema



seq = [-2,1,2,-1,4,-4,-3,3]

changed = True

while changed:

    changed = False
    
    for i in range(len(seq) - 1):
    
        if seq[i] > seq[i+1]:
        
            seq[i], seq[i+1] = seq[i+1], seq[i]
            
            changed = True
            
pass  

### Abordagem do Problema

## Código Python

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import *

def prove(f):
    with Solver(name="z3") as s:
        s.add_assertion(Not(f))
        if s.solve():
            print("Failed to prove.")
        else:
            print("Proved.")

forall i . 0 <= i < tamanho -1 -> (assume seq[i]>seq[i+1]; seq[i] = seq[i] + seq[i+1]; seq[i+1] = seq[i] - seq[i+1]; seq[i] = seq[i] - seq[i+1];);

Pre condição: Variável changed a True
Pos condição: Array ordenado

In [2]:
# Auxiliares
def prime(v):
    return Symbol("next(%s)" % v.symbol_name(), v.symbol_type())
def fresh(v):
    return FreshSymbol(typename=v.symbol_type(),template=v.symbol_name()+"_%d")

# A classe "Sigle Assignment Unfold"
class SAU(object):
    """Trivial representation of a while cycle and its unfolding."""
    def __init__(self, variables, pre , pos, control, trans, sname="z3"):
              
        self.variables = variables       # variables   
        self.pre = pre                   # pre-condition as a predicate in "variables"
        self.pos = pos                   # pos-condition as a predicate in "variables"
        self.control = control           # cycle control as a predicate in "variables"
        self.trans = trans               # cycle body as a binary transition relation 
                                         # in "variables" and "prime variables"
        
        self.prime_variables = [prime(v) for v in self.variables]
        self.frames = [And([Not(control),pos])]  
                 # inializa com uma só frame: a da terminação do ciclo
        
        self.solver = Solver(name=sname)

    def new_frame(self):        
        freshs = [fresh(v) for v in self.variables]    
        b = self.control
        S = self.trans.substitute(dict(zip(self.prime_variables,freshs)))
        W = self.frames[-1].substitute(dict(zip(self.variables,freshs)))
        
        self.frames.append(And([b , ForAll(freshs, Implies(S, W))]))
        
    def unfold(self,bound=0):
        n = 0
        while True:
            if n > bound:
                print("falha: número de tentativas ultrapassa o limite %d "%bound)
                break
            
            f = Or(self.frames)
            if self.solver.solve([self.pre,Not(f)]):  
                self.new_frame()
                n += 1
            else:
                print("sucesso na tentativa %d "%n)
                break   

In [3]:
def trans(seq, seqlinha, tamanho, changedlinha, bubble_up, axioms):

    n = Symbol('n',INT)
    
    assingment = ForAll([n], Equals(Select(seqlinha, n), Ite(Select(seq, n + Int(1)) < bubble_up(n), Select(seq, n + Int(1)), bubble_up(n))))

    change = Implies(ForAll([n],Implies(And(n > 0, n <= tamanho), Equals(Select(seq, n), Select(seqlinha, n)))), Iff(changedlinha, Bool(True)))
                
    return And(axioms, assingment, change)
     
    
    

In [4]:
tamanho = Symbol('tam', INT)

l = [-2,1,2,-1,4,-4,-3,3]

seq = Symbol('seq', ArrayType(INT,INT))

store = And(Equals(Select(seq, Int(0)), Int(l[0])),
            Equals(Select(seq, Int(1)), Int(l[1])),
            Equals(Select(seq, Int(2)), Int(l[2])),
            Equals(Select(seq, Int(3)), Int(l[3])),
            Equals(Select(seq, Int(4)), Int(l[4])),
            Equals(Select(seq, Int(5)), Int(l[5])),
            Equals(Select(seq, Int(6)), Int(l[6])),
            Equals(Select(seq, Int(7)), Int(l[7])),
            )

bubble_up = Symbol('bubble_up', FunctionType(INT,[INT]))
n = Symbol('n',INT)

ax1 = Equals(bubble_up(Int(0)), Select(seq, Int(0)))
ax2 = ForAll([n], Implies(And(n > Int(0), n < tamanho), Equals(bubble_up(n), Ite(Select(seq, n) > bubble_up(n-1), Select(seq, n), bubble_up(n-1)))))
ax3 = Equals(bubble_up(tamanho), Ite(Select(seq, tamanho) > Select(seq, tamanho-Int(1)), Select(seq, tamanho), Select(seq, tamanho-Int(1))))

axioms = And(ax1,ax2,ax3)

m = Symbol('m', INT)

size = 7

changed = Symbol('changed', BOOL)

variables = [changed, tamanho]

pre = And(Iff(changed,Bool(True)), Equals(tamanho,Int(size)), store, axioms)
cond = Iff(changed,Bool(True))
trans = trans(seq, prime(seq), tamanho, prime(changed), bubble_up, axioms)
pos = And(ForAll([m], Implies(And(m > Int(0), m < tamanho), Select(seq, m) < Select(seq, m + Int(1)))), Iff(changed, Bool(False)))


W = SAU(variables,pre,pos,cond,trans)

W.unfold(8)

SolverReturnedUnknownResultError: 